# Preparing the Data

In [1]:
# Set your working directory
from os import getcwd
import pandas as pd
import numpy as np
import pickle as pk
from nltk import stem
import json
import matplotlib.pyplot as plt

fpath = getcwd()
print (fpath)

/Users/pierlim/PycharmProjects/sent_mining_CA


In [3]:
datapath = fpath + '/data/'
yjson = json.load(open(datapath + "yelp_2.json", "rb"))  # there are 35 reviews in this json
reviews = {}
i=0
for key0, val0 in yjson.items():  # keys are 0, 20  etc..
    for val in val0["review"]:
        reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                     "review" : val["description"]}
        i += 1
print ("Total no of reviews " + str(len(reviews)))

Total no of reviews 35


## Reading Scraped Yelp Reviews 

In [41]:
# TODO repeat the above for scraped reviews, then combined both to create a combined dictionary. 

## Create the class labels -1 (neg), 1 (pos)
Note that we should not use lecturer-supplied train_neg.csv and train_pos.csv as the criteria for -ve and +ve may be different. 

We need to build our own training set based on what we scraped as well as the yelp_2.json

Use the ratings 1-3 as negative and 4-5 as positive in the Yelp reviews.

In [26]:
def mapPosNegR(score):
    rate = 1
    if score <= 3: rate = -1   # the result is dependent on the map fn
    return rate

actuals = []
for key, val in reviews.items():  # going through each review
    actuals.append(val['rating'])
    
actuals = list(map(mapPosNegR, actuals))  # actuals in terms of +1 or -1

In [27]:
df_reviews = pd.DataFrame(reviews).transpose()

In [28]:
df_reviews = pd.concat([df_reviews, pd.DataFrame(actuals)], axis=1)
#df_reviews.columns = {'rating', 'review', 'sentiment'}

In [30]:
df_reviews.rename(columns={0:'sentiment'}, 
                 inplace=True)
df_reviews.head()

,rating,review,sentiment
0,5,The service at Sushi Bar was great with an ext...,1
1,2,OMG Yelp let me down big time. I don't know h...,-1
2,4,Heard that this was the best sushi joint in Si...,1
3,3,Ebi fry salmon Aburizal ($17.90++) - 3 stars\n...,-1
4,5,Yes I also get the raving reviews now.\nWe dec...,1


Most reviews here are positive according to our rating criteria: it is an imbalanced set. 

In [38]:
df_reviews.groupby('sentiment').sum()

,rating,review
sentiment,,
-1,11,OMG Yelp let me down big time. I don't know h...
1,141,The service at Sushi Bar was great with an ext...


In [43]:
df_reviews
df_reviews.to_csv('./data/df_reviews.csv', index=False) # TODO this needs to be updated with scraped data 
